# Small Molecule Pipeline

In [1]:
from openpharmmdflow.pipeline.sm.pipeline_settings import *
from openpharmmdflow.pipeline.sm.pipeline import SmallMoleculePipeline
from openff.interchange.components._packmol import UNIT_CUBE, RHOMBIC_DODECAHEDRON
import nglview

First we create the pipeline config settings object

In [25]:
# Any file supported by openff-toolkit or cif
input = {
    "name": "humira",
    "path": "/home/mmh/Projects/OpenPharmMDFlow/openpharmmdflow/tests/io/data/MCL1_no_water.pdb",
}
# input = {"name": "humira", "path": "/home/mmh/Projects/OpenPharmMDFlow/examples/large_molecules/CGMD/humera/20240627_humira_noglyco_ph6.pdb"}


sm_input = SmallMoleculePipelineInputConfig(**input)

settings = {
    # Not used right now
    "work_dir": "/home/mmh/Projects/OpenPharmMDFlow/experiments/sm/scratch",
    "inputs": sm_input,
    "prep_config": SmallMoleculePipelinePrepConfig(
        bespokefit_config=BespokefitConfig(
            bespoke_workflow_factory_config=BespokeWorkflowFactoryConfig(),
            bespoke_executor_config=BespokeExecutorConfig(),
            mol_to_bespoke="humira",
        )
    ),
    "pack_config": SmallMoleculePipelinePackConfig(
        molecule_names=["humira"], number_of_copies=[1], mass_density=0.1
    ),
    "parameterize_config": SmallMoleculePipelineParameterizeConfig(
        force_field="ff14sb_off_impropers_0.0.3.offxml"
    ),
    "simulate_config": SmallMoleculePipelineSimulateConfig(),
    "analyize_config": SmallMoleculePipelineAnalyizeConfig(),
    "solvate_config": SmallMoleculePipelineSolvateConfig(
        nacl_conc=Quantity(0.1, "mole / liter"),
        padding=Quantity(1.2, "nanometer"),
        box_shape=RHOMBIC_DODECAHEDRON,
        target_density=Quantity(0.9, "gram / milliliter"),
        tolerance=Quantity(2.0, "angstrom"),
    ),
}
sm_config = SmallMoleculePipelineConfig(**settings)

## Now we can create our pipeline object

In [26]:
smp = SmallMoleculePipeline(sm_config)

In [27]:
smp.load()

Prep will run bespoke fit

In [28]:
# smp.prep()

In [29]:
smp.pack()

We can view the topology

In [30]:
smp.topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

In [31]:
smp.solvate()

In [33]:
smp.topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

In [ ]:
smp.parameterize()

In [12]:
# smp.interchange.visualize("nglview")

NGLWidget()

In [13]:
smp.simulate()

Starting simulation
Step, volume (nm^3)
0 284.391
500 294.114
1000 299.106
1500 280.749
2000 258.811
2500 243.191
3000 229.999
3500 218.036
4000 212.131
4500 193.469
Elapsed time: 3.07 seconds


In [22]:
from openpharmmdflow.io.load import load_file

In [2]:
# topo = load_file("/home/mmh/Projects/OpenPharmMDFlow/examples/large_molecules/CGMD/humera/20240627_humira_noglyco_ph6.pdb")
topo = load_file(
    "/home/mmh/Projects/OpenPharmMDFlow/openpharmmdflow/tests/io/data/MCL1.pdb"
)

In [3]:
type(topo)

openff.toolkit.topology.molecule.Molecule

In [ ]:
topo.n_molecules

In [15]:
for mol in topo.molecules:
    print(mol.n_atoms)

20258


In [16]:
protein = topo.molecule(0)

In [14]:
type(protein)

openff.toolkit.topology.molecule.Molecule

In [12]:
# protein.visualize(backend="nglview")

In [9]:
from openff.interchange.components._packmol import UNIT_CUBE, RHOMBIC_DODECAHEDRON

In [15]:
RHOMBIC_DODECAHEDRON.shape

(3, 3)

In [14]:
UNIT_CUBE.shape

(3, 3)